In [1]:

# !pip install keras==2.3.1
# !pip install git+https://www.github.com/keras-team/keras-contrib.git

# !pip install patchify    #To install and import other mentioned libraries  in code
# !pip install segmentation_models

from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, Reshape
from tensorflow.keras import backend as k
from keras.layers import BatchNormalization

2023-11-26 20:35:22.433957: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 20:35:22.433993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 20:35:22.435682: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-26 20:35:22.446514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 20:35:23.985414: W tensorflow/compiler/tf2

In [2]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dense, Activation, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, TFBertModel
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split

/home/semcovici/projetos_pesquisa/Political-Bias-Prediction/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ["WANDB_API_KEY"] = "0" ## to silence warning

In [4]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [5]:
# hyperparameters
max_length = 512
batch_size = 128

In [6]:
# Bert Tokenizer
model_multingual = "bert-base-multilingual-uncased"
model_bertimbau_lg = "neuralmind/bert-large-portuguese-cased"
model_bertimbau = "neuralmind/bert-base-portuguese-cased"

model_name = model_bertimbau

tokenizer = BertTokenizer.from_pretrained(model_name)

In [7]:
data_raw = pd.read_parquet('../../dataset/processed/artigos_tratados/artigos_tratados.parquet')

In [8]:
# tira as linhas sem conteudo
data_raw = data_raw[data_raw['Conteudo'] != '']

X = data_raw.drop(['URL', 'Vies'], axis = 1)
y = data_raw['Vies']

y = pd.get_dummies(y).values

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=X['Partido'])
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.10, random_state=42, stratify=X_train_val['Partido'])

In [9]:
X_train = X_train.drop(['Partido'],axis=1).values

X_test = X_test.drop(['Partido'],axis=1).values

X_val = X_val.drop(['Partido'],axis=1).values

# transforma em uma lista
X_train = X_train.reshape(X_train.shape[0])
X_test = X_test.reshape(X_test.shape[0])
X_val = X_val.reshape(X_val.shape[0])

In [10]:
def bert_encode(data):
    tokens = tokenizer.batch_encode_plus(data, max_length=max_length, padding='max_length', truncation=True)
    
    return tf.constant(tokens['input_ids'])

In [11]:
train_encoded = bert_encode(X_train)

2023-11-26 20:36:13.057120: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-26 20:36:13.110399: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-26 20:36:13.110703: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [12]:
dev_encoded = bert_encode(X_val)

In [13]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_encoded, y_train))
    .shuffle(100)
    .batch(batch_size)
)

dev_dataset = (
    tf.data.Dataset
    .from_tensor_slices((dev_encoded, y_val))
    .shuffle(100)
    .batch(batch_size)
)

In [18]:
def nn_model():
    model= Sequential()

   
    model.add(Dense(20,input_dim=500, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())


    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    
    model.add(Dense(3))
    model.add(Activation('softmax'))

    # compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    
    return model


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Activation, BatchNormalization, Conv1D, MaxPooling1D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import keras
import numpy as np

def build_lstm_model_2(input_shape, output_shape=1):
    keras.backend.clear_session()
    
    model = Sequential()

    # Adicione uma camada de remodelagem para transformar a entrada em 3D
    model.add(Reshape((input_shape, 1), input_shape=(input_shape,)))

    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())

    model.add(Dense(20, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())    
    
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(100, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    
    model.add(Dense(output_shape, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(1e-4),
                  metrics=['accuracy'])
    
    return model

# Supondo que você tenha um conjunto de treinamento (train_dataset) e um conjunto de validação (dev_dataset)

model = build_lstm_model_2(input_shape=512, output_shape=3)

# Adicionando callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Treinamento do modelo
history = model.fit(
    train_dataset,
    epochs=100,
    callbacks=[early_stopping, reduce_lr],
    validation_data=dev_dataset,
    batch_size=batch_size,
    shuffle=True,
)


Epoch 1/100


2023-11-26 20:37:26.147128: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2023-11-26 20:37:27.412845: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc730bdd250 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-26 20:37:27.412861: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2023-11-26 20:37:27.417826: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1701041847.508236   17712 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


63/63 [==============================] - 49s 653ms/step - loss: 5.7991 - accuracy: 0.3435 - val_loss: 5.3984 - val_accuracy: 0.2305 - lr: 1.0000e-04
Epoch 2/100
63/63 [==============================] - 43s 686ms/step - loss: 5.5761 - accuracy: 0.4133 - val_loss: 5.3870 - val_accuracy: 0.2305 - lr: 1.0000e-04
Epoch 3/100
63/63 [==============================] - 42s 669ms/step - loss: 5.4890 - accuracy: 0.4598 - val_loss: 5.3639 - val_accuracy: 0.2463 - lr: 1.0000e-04
Epoch 4/100
63/63 [==============================] - 49s 784ms/step - loss: 5.4201 - accuracy: 0.4802 - val_loss: 5.3064 - val_accuracy: 0.5458 - lr: 1.0000e-04
Epoch 5/100
63/63 [==============================] - 48s 767ms/step - loss: 5.3775 - accuracy: 0.5003 - val_loss: 5.2475 - val_accuracy: 0.5424 - lr: 1.0000e-04
Epoch 6/100
63/63 [==============================] - 51s 804ms/step - loss: 5.3410 - accuracy: 0.5060 - val_loss: 5.2113 - val_accuracy: 0.5638 - lr: 1.0000e-04
Epoch 7/100
63/63 [===========================

In [ ]:


# def build_lstm_model_2(input_shape, output_shape=1):
#     keras.backend.clear_session()
    
#     model = Sequential()

#     # Adicione uma camada de remodelagem para transformar a entrada em 3D
#     model.add(Reshape((input_shape, 1), input_shape=(input_shape,)))

#     model.add(Dense(20, activation='relu'))
#     model.add(Dropout(0.2))
#     #model.add(BatchNormalization())
    
#     model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, activation='relu')))
#     model.add(Dropout(0.2))
#     #model.add(BatchNormalization())    
    
#     model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu')))
#     model.add(Dropout(0.2))
#     #model.add(BatchNormalization())

#     model.add(Dense(100, activation='relu'))
#     model.add(Dropout(0.2))
#     #model.add(BatchNormalization())
    
#     model.add(Dense(output_shape))
#     model.add(Activation('softmax'))
    
#     model.compile(loss='categorical_crossentropy',
#                   optimizer=tf.keras.optimizers.Adam(1e-4),
#                   metrics=['accuracy'])
    
#     return model

# model = build_lstm_model_2(input_shape=512, output_shape=3)
# callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# history = model.fit(
#                     train_dataset,
#                     epochs=100,
#                     callbacks=[callback],
#                     validation_data=dev_dataset,
#                     batch_size=32,
#                     shuffle = True,
#                     ) 

Epoch 1/1000
 34/249 [===>..........................] - ETA: 7:09 - loss: nan - accuracy: 0.3208

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test_encoded = bert_encode(test.text)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_encoded)
    .batch(batch_size)
)

predicted_tweets = model.predict(test_dataset, batch_size=batch_size)
predicted_tweets_binary = tf.cast(tf.round(predicted_tweets), tf.int32).numpy().flatten()

my_submission = pd.DataFrame({'id': test.id, 'target': predicted_tweets_binary})
my_submission.to_csv('/kaggle/working/my_submission.csv', index=False)